In [1]:
import json
import pandas as pd

In [2]:
path="2023_7_bike_usage_history.csv"
df = pd.read_csv(path,index_col=0)

In [3]:
df.columns=['lend_time','lend_station_name','return_time','return_station_name','usage_time','source_date']

In [4]:
df['source_date'] = pd.to_datetime(df['source_date'])
df['lend_date'] = pd.to_datetime(df['lend_time']).dt.date
df['lend_hour'] = pd.to_datetime(df['lend_time']).dt.hour
df['return_date'] = pd.to_datetime(df['return_time']).dt.date
df['return_hour'] = pd.to_datetime(df['return_time']).dt.hour

In [5]:
df = df.drop(['lend_time','return_time'],axis=1)
df['usage_time'] = pd.to_timedelta(df['usage_time']).dt.total_seconds().astype('int')

In [6]:
tpe_stations = pd.read_csv('TPE_bike_station.csv')
nwt_stations = pd.read_csv('NWT_bike_station.csv')

In [7]:
tpe_stations = tpe_stations[['station_no','name_tw','district_tw']]
nwt_stations = nwt_stations[['station_no','name_tw','district_tw']]

In [8]:
tpe_stations = tpe_stations.add_prefix('lend_')
nwt_stations = nwt_stations.add_prefix('return_')

In [9]:
df = df.merge(tpe_stations,how='inner',left_on='lend_station_name',right_on='lend_name_tw')

In [10]:
df = df.merge(nwt_stations,how='inner',left_on='return_station_name',right_on='return_name_tw')

In [11]:
df = df[(df['lend_date']==df['return_date']) & (df['lend_hour']==df['return_hour'])]

In [12]:
df = df[['lend_date','lend_hour','lend_station_no','lend_station_name',
    'lend_district_tw','return_station_no','return_station_name',
    'return_district_tw']]

In [13]:
by_station = df.groupby(by=df.columns.to_list(),as_index=False).size()

In [14]:
by_station['day_of_week'] = pd.to_datetime(by_station['lend_date']).dt.day_of_week
by_station['weekend'] = pd.to_datetime(by_station['lend_date']).dt.weekday.isin([5,6])

In [15]:
by_station = by_station.rename(
    {'lend_date':'date',
     'lend_hour':'hour',
     'lend_station_no':'lend_station_id',
     'lend_district_tw':'lend_station_district',
     'return_station_no':'return_station_id',
     'return_district_tw':'return_station_district',
     'size':'traffic_count'},axis=1
)
by_station = by_station[['date','hour','day_of_week','weekend',
                        'lend_station_id','lend_station_name',
                        'lend_station_district','return_station_id',
                        'return_station_name','return_station_district',
                        'traffic_count']]

In [107]:
by_station.to_csv('by_station_single_month.csv',index=False)

In [17]:
by_time = df.groupby(by=['lend_date', 'lend_hour'],as_index=False).size()

In [18]:
by_time['day_of_week'] = pd.to_datetime(by_time['lend_date']).dt.day_of_week
by_time['weekend'] = pd.to_datetime(by_time['lend_date']).dt.weekday.isin([5,6])

In [19]:
by_time = by_time.rename(
    {'lend_date':'date',
     'lend_hour':'hour',
     'size':'traffic_count'},axis=1)
by_time = by_time[['date','hour','day_of_week','weekend','traffic_count']]

In [20]:
by_time

,date,hour,day_of_week,weekend,traffic_count
0,2023-07-01,0,5,True,53
1,2023-07-01,1,5,True,41
2,2023-07-01,2,5,True,19
3,2023-07-01,3,5,True,21
4,2023-07-01,4,5,True,9
...,...,...,...,...,...
739,2023-07-31,19,0,False,115
740,2023-07-31,20,0,False,84
741,2023-07-31,21,0,False,70
742,2023-07-31,22,0,False,55


In [113]:
by_time.to_csv('by_time_single_month.csv',index=False)